- ### Planteamiento del problema:

Predecir qué películas podrían ser o no un éxito comercial.

In [53]:
# Se importan los dos datasets
import pandas as pd

url1 = 'https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv'
dataset1= pd.read_csv(url1)

url2 = 'https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv'
dataset2= pd.read_csv(url2)

- ### Exploración y limpieza de datos:

In [54]:
# Primera apreciacion del dataset1
filas1 = dataset1.shape[0]
columnas1 = dataset1.shape[1]
print(f'Filas: {filas1} | Columnas: {columnas1}')
print()
print(dataset1.info())

Filas: 4803 | Columnas: 20

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object

In [55]:
# Primera apreciacion del dataset2
filas2 = dataset2.shape[0]
columnas2 = dataset2.shape[1]
print(f'Filas: {filas2} | Columnas: {columnas2}')
print()
print(dataset2.info())

Filas: 4803 | Columnas: 4

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB
None


Creando la base de datos

In [56]:
import sqlite3

connection = sqlite3.connect("movies.db")

In [57]:
# Guardando ambos datasets en la base de datos

dataset1.to_sql('movies', connection, index = False, if_exists = 'replace')
dataset2.to_sql('credits', connection, index = False, if_exists = 'replace')

4803

In [58]:
#Uniendo ambas tablas

query = '''
    SELECT m.id AS movie_id, 
           m.title, 
           m.overview, 
           m.genres, 
           m.keywords, 
           c.cast, 
           c.crew
    FROM movies m
    JOIN credits c ON m.title = c.title
'''

In [59]:
import pandas as pd

dataframe = pd.read_sql_query(sql=query, con = connection)

In [60]:
# Cerrar la conexión a la base de datos
connection.close()

In [61]:
dataframe.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [62]:
import json

dataframe['genres'] = dataframe['genres'].apply(lambda x: [item['name'] for item in json.loads(x)])
dataframe['keywords'] = dataframe['keywords'].apply(lambda x: [item['name'] for item in json.loads(x)])
dataframe['cast'] = dataframe['cast'].apply(lambda x: [item['name'] for item in json.loads(x)][:3])

In [63]:
dataframe.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [64]:
dataframe.crew[0]

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [5]:
nulos = dataframe.isnull().sum()

if len(nulos[nulos != 0]) == 0:

    print('No hay datos nulos')

else:

    print(nulos[nulos != 0])

No hay datos nulos


In [6]:
dataframe.dtypes.value_counts()

object    2
int64     1
Name: count, dtype: int64

- Inicialmente se puede observar que no existen valores nulos en ninguna de las variables.
- Hay 891 registros y 3 variables incluyendo la variable objetivo.
- Se establece la variable objetivo a "polarity".

In [7]:
# Se verifica la existencia de duplicados
duplicados = dataframe.duplicated().sum()
print(f'Existen {duplicados} registros duplicados.')
print(f'El % de registros "duplicados" es {duplicados/filas}.')

Existen 0 registros duplicados.
El % de registros "duplicados" es 0.0.


No Existen datos duplicados en el dataframe.

Se elimina columna "package_name" ya que para calificar el comentario es irrelevante desde qué app se realizó el mismo. Sólo importa el contenido del comentario en sí mismo.

In [8]:
dataframe.drop(['package_name'], axis = 1, inplace = True)

Todas las palabras en minusculas y eliminar espacios antes y despues de cada cadena de caracteres.

In [9]:
dataframe["review"] = dataframe["review"].str.strip().str.lower()

In [10]:
dataframe.review

0      privacy at least put some option appear offlin...
1      messenger issues ever since the last update, i...
2      profile any time my wife or anybody has more t...
3      the new features suck for those of us who don'...
4      forced reload on uploading pic on replying com...
                             ...                        
886    loved it i loooooooooooooovvved it because it ...
887    all time legendary game the birthday party lev...
888    ads are way to heavy listen to the bad reviews...
889    fun works perfectly well. ads aren't as annoyi...
890    they're everywhere i see angry birds everywher...
Name: review, Length: 891, dtype: object

Dividir train y test data

In [11]:
from sklearn.model_selection import train_test_split

X = dataframe["review"]
y = dataframe["polarity"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

X_train.head()

331    just did the latest update on viber and yet ag...
733    keeps crashing it only works well in extreme d...
382    the fail boat has arrived the 6.0 version is t...
704    superfast, just as i remember it ! opera mini ...
813    installed and immediately deleted this crap i ...
Name: review, dtype: object

Matriz de conteo de palabras

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

vec_model = CountVectorizer(stop_words = "english")
X_train = vec_model.fit_transform(X_train).toarray()
X_test = vec_model.transform(X_test).toarray()

X_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

- #### Entrenamiento del modelo

Primero se evalúan los modelos base GaussianNB, MultinomialNB y BernoulliNB

In [13]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

modelos = {
    'GNB': GaussianNB(),
    'MNB': MultinomialNB(),
    'BNB': BernoulliNB()
}

In [14]:
def prueba_modelos(modelo, X_train, X_test, y_train, y_test):

    resultados_modelos = {}

    for name, model in modelos.items():

        model.fit(X_train, y_train)

        resultados_modelos[name] = model.score(X_test, y_test)

    return resultados_modelos

In [15]:
prueba_modelos(modelo=modelos, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)

{'GNB': 0.8044692737430168,
 'MNB': 0.8156424581005587,
 'BNB': 0.770949720670391}

Se optimizará el modelo Multinomial por ser el que mejor precisión dio.

In [21]:
hyperparams = {
    'alpha' : np.linspace(0,5,200),
    'force_alpha' : [True, False],
    'fit_prior' : [True, False],
}

In [22]:
from sklearn.model_selection import GridSearchCV

modelo_grid = GridSearchCV(estimator=MultinomialNB(),
                           param_grid=hyperparams,
                           scoring='accuracy',
                           n_jobs=-1,
                           cv=5                        
)

In [23]:
modelo_grid.fit(X_train, y_train)

/workspaces/EDA/EDA-env/lib/python3.11/site-packages/sklearn/naive_bayes.py:890: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = np.log(smoothed_fc) - np.log(
/workspaces/EDA/EDA-env/lib/python3.11/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/workspaces/EDA/EDA-env/lib/python3.11/site-packages/sklearn/naive_bayes.py:890: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = np.log(smoothed_fc) - np.log(
/workspaces/EDA/EDA-env/lib/python3.11/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/workspaces/EDA/EDA-env/lib/python3.11/site-packages/sklearn/naive_bayes.py:890: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = np.log(smoothed_fc) - np.log(
/workspaces/EDA/EDA-env/lib/python3.11/site-packages/sklearn/utils/extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  r

GridSearchCV(cv=5, estimator=MultinomialNB(), n_jobs=-1,
             param_grid={'alpha': array([0.        , 0.02512563, 0.05025126, 0.07537688, 0.10050251,
       0.12562814, 0.15075377, 0.1758794 , 0.20100503, 0.22613065,
       0.25125628, 0.27638191, 0.30150754, 0.32663317, 0.35175879,
       0.37688442, 0.40201005, 0.42713568, 0.45226131, 0.47738693,
       0.50251256, 0.52763819, 0.55276382, 0.57788945, 0.60301508,
       0.6...
       4.27135678, 4.29648241, 4.32160804, 4.34673367, 4.3718593 ,
       4.39698492, 4.42211055, 4.44723618, 4.47236181, 4.49748744,
       4.52261307, 4.54773869, 4.57286432, 4.59798995, 4.62311558,
       4.64824121, 4.67336683, 4.69849246, 4.72361809, 4.74874372,
       4.77386935, 4.79899497, 4.8241206 , 4.84924623, 4.87437186,
       4.89949749, 4.92462312, 4.94974874, 4.97487437, 5.        ]),
                         'fit_prior': [True, False],
                         'force_alpha': [True, False]},
             scoring='accuracy')

In [31]:
modelo_grid.best_params_

{'alpha': 1.8341708542713568, 'fit_prior': False, 'force_alpha': True}

In [39]:
model_opti = MultinomialNB(alpha = 1.8341708542713568, fit_prior = False, force_alpha = True)

model_opti.fit(X_train, y_train)

MultinomialNB(alpha=1.8341708542713568, fit_prior=False)

In [40]:
y_predict_best = model_opti.predict(X_test)

In [41]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_predict_best)
print('Precisión')
print(round(accuracy,6))

Precisión
0.815642


No mejoró el rendimiento con el proceso de GridSearch, para los parámetros estudiados.

In [39]:
from pickle import dump

dump(model_opti, open("../models/naive_bayes.sav", "wb"))